# Detect driver's face and facial landmarks

In [1]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/driver-videos/videos/P1043117_720.mp4
/kaggle/input/driver-videos/videos/P1043134_720.mp4
/kaggle/input/driver-videos/videos/P1043131_720.mp4
/kaggle/input/driver-videos/videos/P1043114_720.mp4
/kaggle/input/driver-videos/videos/P1043075_720.mp4
/kaggle/input/driver-videos/videos/P1043124_720.mp4
/kaggle/input/driver-videos/videos/P1043119_720.mp4
/kaggle/input/driver-videos/videos/P1043129_720.mp4
/kaggle/input/driver-videos/videos/P1043132_720.mp4
/kaggle/input/driver-videos/videos/P1043133_720.mp4
/kaggle/input/driver-videos/videos/P1042786_720.mp4
/kaggle/input/driver-videos/videos/P1042793_720.mp4
/kaggle/input/driver-videos/videos/P1042797_720.mp4
/kaggle/input/driver-videos/videos/P1042772_720.mp4
/kaggle/input/driver-videos/videos/P1043080_720.mp4
/kaggle/input/driver-videos/videos/P1043078_720.mp4
/kaggle/input/driver-videos/videos/P1043123_720.mp4
/kaggle/input/driver-videos/videos/P1042751_720.mp4
/kaggle/input/driver-videos/videos/P1043068_720.mp4
/kaggle/inpu

In [2]:
!pip install  retinaface_pytorch > /dev/null
!mkdir frames

In [3]:
import os
import json
from tqdm import tqdm
import cv2
import numpy as np
from matplotlib import pyplot as plt
from retinaface.pre_trained_models import get_model
from retinaface.utils import vis_annotations

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
# Extract frames from all videos in current directory
dir = "/kaggle/input/driver-videos/videos/"
for file in os.listdir(dir):
    if "mp4" in file:
        print(f"Extracting frames, video: {file}")
        if not os.path.exists(f"./frames/{file.split('.')[-2]}"):
            os.mkdir(f"./frames/{file.split('.')[-2]}")

        count = 0
        vidcap = cv2.VideoCapture(os.path.join(dir, file))
        success, img = vidcap.read()
        while success:
            cv2.imwrite(f"./frames/{file.split('.')[-2]}/frame{count}.jpg", img)
            success, img = vidcap.read()
            count += 1
            
        print(f"Frames extracted: {count}")

Extracting frames, video: P1043117_720.mp4
Frames extracted: 480
Extracting frames, video: P1043134_720.mp4
Frames extracted: 635
Extracting frames, video: P1043131_720.mp4
Frames extracted: 670
Extracting frames, video: P1043114_720.mp4
Frames extracted: 350
Extracting frames, video: P1043075_720.mp4
Frames extracted: 3285
Extracting frames, video: P1043124_720.mp4
Frames extracted: 610
Extracting frames, video: P1043119_720.mp4
Frames extracted: 480
Extracting frames, video: P1043129_720.mp4
Frames extracted: 3095
Extracting frames, video: P1043132_720.mp4
Frames extracted: 705
Extracting frames, video: P1043133_720.mp4
Frames extracted: 705
Extracting frames, video: P1042786_720.mp4
Frames extracted: 3275
Extracting frames, video: P1042793_720.mp4
Frames extracted: 3145
Extracting frames, video: P1042797_720.mp4
Frames extracted: 3145
Extracting frames, video: P1042772_720.mp4
Frames extracted: 3105
Extracting frames, video: P1043080_720.mp4
Frames extracted: 550
Extracting frames, 

In [5]:
def max_bbox(preds):
    """Return idx of face with max area of bbox."""

    def compute_area(bbox):
        return (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])

    return np.argmax([compute_area(pred["bbox"]) for pred in preds])


def annotate_sequence(seq_dir, model):
    """Iterate over frames in video sequence and annotate bbox 
    and facial landmarks of the driver."""

    annotations = {}
    for frame in tqdm(os.listdir(seq_dir)):
        if ".jpg" in frame:
            img = cv2.imread(os.path.join(seq_dir, frame))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            pred = model.predict_jsons(img)
            pred = ([p for p in pred if p["score"] != -1])

            if len(pred) == 0:
                continue

            max_bbox_idx = max_bbox(pred)
            pred = pred[max_bbox_idx]

            # plt.imshow(vis_annotations(img, [pred]))
            # plt.show()
            
            annotations[frame] = {}
            annotations[frame]["bbox"] = pred["bbox"]
            annotations[frame]["landmarks"] = pred["landmarks"]

    return annotations

In [6]:
dir_list = "./frames/"
model = get_model("resnet50_2020-07-20", max_size=2048, device="cuda")
model.eval()

# loop through each item in the directory list
for seq in os.listdir(dir_list):
    print(f"Annotating seq: {seq}")
    
    annots = annotate_sequence(os.path.join(dir_list, seq), model)
    with open(f"{os.path.join(dir_list, seq)}/annotations.json", "w") as file:
        json.dump(annots, file)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://github.com/ternaus/retinaface/releases/download/0.01/retinaface_resnet50_2020-07-20-f168fae3c.zip" to /root/.cache/torch/hub/checkpoints/retinaface_resnet50_2020-07-20-f168fae3c.zip
100%|██████████| 96.9M/96.9M [00:01<00:00, 63.1MB/s]
/opt/conda/lib/python3.10/site-packages/torch/hub.py:665: UserWarning: Falling back to the old format < 1.6. This support will be deprecated in favor of default zipfile format introduced in 1.6. Please redo torch.save() to save it in 

Annotating seq: P1043106_720


100%|██████████| 3100/3100 [10:36<00:00,  4.87it/s]


Annotating seq: P1043078_720


100%|██████████| 588/588 [02:01<00:00,  4.85it/s]


...

100%|██████████| 3275/3275 [11:25<00:00,  4.78it/s]


Annotating seq: P1043120_720


100%|██████████| 418/418 [01:26<00:00,  4.81it/s]


Annotating seq: P1043089_720


100%|██████████| 3120/3120 [10:52<00:00,  4.78it/s]
